In [2]:
import sys
sys.path.insert(0, "/home/fcdl/Develop/ICL/")
from data.iCIFAR import ICIFAR
from data.idadataloader import IDADataloader 
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
import torch
#import matplotlib.pyplot as plt
import numpy as np
from data.common import DatasetPrototypes


In [3]:
# Chose the dataset to test!
dataset = 'cifar'
# dataset = 'idad'

In [4]:
def make_protos(idx):
    proto_x = []
    proto_y = []

    for i in idx:
        img = data.get_images_of_class(i)
        proto_x += img
        proto_y += [i for j in range(len(img))]
    
    return proto_x, proto_y

num_cl_first = 5
num_cl_after = num_cl_first

# ICIFAR

In [5]:
if dataset == 'cifar':
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(ICIFAR.MEAN, ICIFAR.STD)])

    augmentation = transforms.RandomCrop((32,32), padding=4)

    data = ICIFAR('/home/fcdl/dataset', download=False, num_cl_first=num_cl_first, num_cl_after=num_cl_after, 
                  augmentation=None, transform=transform, batch_size=64, run_number=0, workers=8)
    # Show example image
    data.train_dataset[0][0].show()

In [11]:
tr = transforms.Compose([transforms.ToPILImage(),augmentation])

tr(data.valid_dataset[0][0]).show()

# IDAD

In [18]:
if dataset == 'idad':
    transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
    augmentation = transforms.Compose([transforms.Resize(400), transforms.RandomCrop(224)])

    targ = ImageFolder('/home/fcdl/dataset/office/Art')
    src = ImageFolder('/home/fcdl/dataset/office/Product')
    data = IDADataloader(targ, src, 
                      num_cl_first=num_cl_first, num_cl_after=num_cl_after, 
                      augmentation=augmentation, transform=transform, 
                      batch_size=64, run_number=0, workers=8)
    # Show example image from target
    data.target[0][0].show()
    # Show example image from source
    data.source[0][0].show()

# TEST!

In [4]:
# Check if get dataloader returns the same images and in the same order as get images
# And, moreover, that images are returned as tensor not normalized or augmented
# That dataloader and prototypes are normalized in the same way

proto_set = DatasetPrototypes(*make_protos([19]), transform) # same as applying the target transform to proto

dl = data.get_dataloader_of_class(19)
# reconstruct the list of images from dataloader
ll = []
for img, tar in dl:
    ll.append(img)

images = torch.cat(ll)

flag = True
for i in range(len(proto_set)):
    if not torch.all(torch.eq(images[i], proto_set[i][0])):
        flag = False
        
print(len(proto_set))
if flag:
    print("Same order!")
else:
    assert False, "Not same order"

#proto_x[0].show()
#print(proto_set[0][0])
#print(images[0])

58
Same order!


In [25]:
# check if dataloader of training works
first = True
#first = False

if first:
    data.reset_iteration()
    iteration = 0
else:
    iteration += 1
    
order = [x.item() for x in data.order[data.offset(iteration-1): data.offset(iteration)]]
print(f"Class should be\n{sorted(order)}")

data_loader = data.next_iteration()

print(f"Classes are")
count = 0
first = True
for inp, target in data_loader:
    count += inp.shape[0]
    if first:
        print([x.item() for x in target.unique().sort()[0]])
        first = False     

# for easiness of testing, set num_cl_first = num_cl_after
print(count)
if dataset == 'cifar':
    assert count == 500  * num_cl_first, "Not all samples are returned"
elif dataset == 'idad':
    assert num_cl_first*50 < count < num_cl_first*60, "Not all samples are returned"

print("\nOK")

Class should be
[0, 1, 2, 3, 4]
Classes are
[0, 1, 2, 3, 4]
257

OK


In [28]:
# check if dataloader of training works
first = True
idx = [27, 29]

if first:
    data.reset_iteration()
    iteration = 0
else:
    iteration += 1
    
order = [x.item() for x in data.order[data.offset(iteration-1): data.offset(iteration)]]
print(f"Class should be\n{sorted(order + idx)}")

proto_len = len(make_protos(idx)[0])
data_loader = data.next_iteration(*make_protos(idx))

print(f"Classes are")
count = 0
first = True
for inp, target in data_loader:
    count += inp.shape[0]
    if first:
        print([x.item() for x in target.unique().sort()[0]])
        first = False

# for easiness of testing, set num_cl_first = num_cl_after 
n_c = num_cl_first+len(idx)
print(count)
if dataset == 'cifar':
    assert count == 500  * (n_c), "Not all samples are returned"
elif dataset == 'idad':
    assert num_cl_first*50 < count - proto_len < num_cl_first*60, "Not all samples are returned"
    assert proto_len == 189, "Please, set proto idx to 27, 29"

print("\nOK")

Class should be
[0, 1, 2, 3, 4, 27, 29]
Classes are
[0, 1, 2, 3, 4, 27, 29]
446

OK


In [35]:
# check if dataloader of validation works
#first = True
first = False

if first:
    data.reset_iteration()
    iteration = 0
else:
    iteration += 1
    
order = [x.item() for x in data.order[0: data.offset(iteration)]]
print(f"Class should be\n{sorted(order)}")

dl = data.test_dataloader(iteration=iteration)

count = 0
classes = []
for inp, target in dl:
    count += inp.shape[0]
    classes += [x.item() for x in target.unique().sort()[0]]

classes = sorted(list(set(classes)))

print(f"They are\n{classes}")

# for easiness of testing, set num_cl_first = num_cl_after 
n_c = num_cl_first*(iteration+1)
print(count)
if dataset == 'cifar':
    assert count == 100  * (n_c), "Not all samples are returned"
elif dataset == 'idad':
    assert n_c*40 < count < n_c*60, "Not all samples are returned"

print("\nOK")

Class should be
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
They are
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
1089

OK
